#                 ---Financial Fraud Detection---

In [17]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [18]:
# load the dataset
data = pd.read_csv('C:\\Users\\user\\Downloads\\Bank_Transaction_Fraud_Detection.csv')
data.head()

,Customer_ID,Customer_Name,Gender,Age,State,City,Bank_Branch,Account_Type,Transaction_ID,Transaction_Date,...,Merchant_Category,Account_Balance,Transaction_Device,Transaction_Location,Device_Type,Is_Fraud,Transaction_Currency,Customer_Contact,Transaction_Description,Customer_Email
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,23-01-2025,...,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,+9198579XXXXXX,Bitcoin transaction,oshaXXXXX@XXXXX.com
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,11-01-2025,...,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,+9191074XXXXXX,Grocery delivery,hredhaanXXXX@XXXXXX.com
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,25-01-2025,...,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,+9197745XXXXXX,Mutual fund investment,ekaniXXX@XXXXXX.com
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,19-01-2025,...,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,+9195889XXXXXX,Food delivery,yaminiXXXXX@XXXXXXX.com
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,30-01-2025,...,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,+9195316XXXXXX,Debt repayment,kritikaXXXX@XXXXXX.com


### Data Preprocessing

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 24 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              200000 non-null  object 
 1   Customer_Name            200000 non-null  object 
 2   Gender                   200000 non-null  object 
 3   Age                      200000 non-null  int64  
 4   State                    200000 non-null  object 
 5   City                     200000 non-null  object 
 6   Bank_Branch              200000 non-null  object 
 7   Account_Type             200000 non-null  object 
 8   Transaction_ID           200000 non-null  object 
 9   Transaction_Date         200000 non-null  object 
 10  Transaction_Time         200000 non-null  object 
 11  Transaction_Amount       200000 non-null  float64
 12  Merchant_ID              200000 non-null  object 
 13  Transaction_Type         200000 non-null  object 
 14  Merc

In [20]:
# change data type of date column to datetime
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])
data['Transaction_Time'] = pd.to_datetime(data['Transaction_Time'], format='%H:%M:%S').dt.time
# combine date and time into a single datestamp column
data['Transaction_Timestamp'] = pd.to_datetime(
    data['Transaction_Date'].astype(str) + ' ' + data['Transaction_Time'].astype(str)
)
# drop original date and time columns
data.drop(columns=['Transaction_Date', 'Transaction_Time'], inplace=True)
data.info()

C:\Users\user\AppData\Local\Temp\ipykernel_16736\2201368569.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 23 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Customer_ID              200000 non-null  object        
 1   Customer_Name            200000 non-null  object        
 2   Gender                   200000 non-null  object        
 3   Age                      200000 non-null  int64         
 4   State                    200000 non-null  object        
 5   City                     200000 non-null  object        
 6   Bank_Branch              200000 non-null  object        
 7   Account_Type             200000 non-null  object        
 8   Transaction_ID           200000 non-null  object        
 9   Transaction_Amount       200000 non-null  float64       
 10  Merchant_ID              200000 non-null  object        
 11  Transaction_Type         200000 non-null  object        
 12  Merchant_Categor

In [21]:
# check duplicate rows on a specific subset of columns
data.duplicated(subset=['Transaction_ID', 'Customer_ID', 'Transaction_Amount', 'Transaction_Timestamp']).sum()

np.int64(0)

In [22]:
# drop unnecessary columns
data.drop(columns=['Customer_Email', 'Customer_Contact'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 21 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Customer_ID              200000 non-null  object        
 1   Customer_Name            200000 non-null  object        
 2   Gender                   200000 non-null  object        
 3   Age                      200000 non-null  int64         
 4   State                    200000 non-null  object        
 5   City                     200000 non-null  object        
 6   Bank_Branch              200000 non-null  object        
 7   Account_Type             200000 non-null  object        
 8   Transaction_ID           200000 non-null  object        
 9   Transaction_Amount       200000 non-null  float64       
 10  Merchant_ID              200000 non-null  object        
 11  Transaction_Type         200000 non-null  object        
 12  Merchant_Categor

In [23]:
# covert date and time format to US format
data['Transaction_Timestamp'] = data['Transaction_Timestamp'].dt.strftime('%m/%d/%Y %I:%M:%S %p')
data['Transaction_Timestamp'].head()

0    01/23/2025 04:04:07 PM
1    01/11/2025 05:14:53 PM
2    01/25/2025 03:09:52 AM
3    01/19/2025 12:27:02 PM
4    01/30/2025 06:30:46 PM
Name: Transaction_Timestamp, dtype: object

In [24]:
# rename columns for better readability
data.columns = [col.strip().replace(' ', '_').lower() for col in data.columns]
data.head()

,customer_id,customer_name,gender,age,state,city,bank_branch,account_type,transaction_id,transaction_amount,...,transaction_type,merchant_category,account_balance,transaction_device,transaction_location,device_type,is_fraud,transaction_currency,transaction_description,transaction_timestamp
0,d5f6ec07-d69e-4f47-b9b4-7c58ff17c19e,Osha Tella,Male,60,Kerala,Thiruvananthapuram,Thiruvananthapuram Branch,Savings,4fa3208f-9e23-42dc-b330-844829d0c12c,32415.45,...,Transfer,Restaurant,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",POS,0,INR,Bitcoin transaction,01/23/2025 04:04:07 PM
1,7c14ad51-781a-4db9-b7bd-67439c175262,Hredhaan Khosla,Female,51,Maharashtra,Nashik,Nashik Branch,Business,c9de0c06-2c4c-40a9-97ed-3c7b8f97c79c,43622.60,...,Bill Payment,Restaurant,74622.66,POS Mobile Device,"Nashik, Maharashtra",Desktop,0,INR,Grocery delivery,01/11/2025 05:14:53 PM
2,3a73a0e5-d4da-45aa-85f3-528413900a35,Ekani Nazareth,Male,20,Bihar,Bhagalpur,Bhagalpur Branch,Savings,e41c55f9-c016-4ff3-872b-cae72467c75c,63062.56,...,Bill Payment,Groceries,66817.99,ATM,"Bhagalpur, Bihar",Desktop,0,INR,Mutual fund investment,01/25/2025 03:09:52 AM
3,7902f4ef-9050-4a79-857d-9c2ea3181940,Yamini Ramachandran,Female,57,Tamil Nadu,Chennai,Chennai Branch,Business,7f7ee11b-ff2c-45a3-802a-49bc47c02ecb,14000.72,...,Debit,Entertainment,58177.08,POS Mobile App,"Chennai, Tamil Nadu",Mobile,0,INR,Food delivery,01/19/2025 12:27:02 PM
4,3a4bba70-d9a9-4c5f-8b92-1735fd8c19e9,Kritika Rege,Female,43,Punjab,Amritsar,Amritsar Branch,Savings,f8e6ac6f-81a1-4985-bf12-f60967d852ef,18335.16,...,Transfer,Entertainment,16108.56,Virtual Card,"Amritsar, Punjab",Mobile,0,INR,Debt repayment,01/30/2025 06:30:46 PM
